In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [2]:
df = pd.read_csv('../datasets/cleaned_data.csv')

In [3]:
# df.fillna(0, inplace=True)
df.columns

Index(['Country Name', 'Country Code', 'rural_pop_percent_17',
       'food_production_index_17', 'ag_land_area_17',
       'arable_land_percent_2017', 'net_migration_2017', 'hiv_prevalence_2017',
       'mat_mortality_ratio_2017', 'under5_mortality_ratio_2017',
       'tubercul_incidence_2017', 'elec_access_2017',
       'ren_energy_percent_2017', 'ffuel_energy_percent_2017',
       'co2_emissions_2017', 'pop_air_pollution_2017', 'foreign_dir_inv_2017',
       'atm_access_2017', 'adol_fertility_rate_2017',
       'fem_labor_part_rate_2017', 'male_labor_part_rate_2017',
       'fertility_rate_2017', 'dpt_immuniz_rate_2017',
       'undernourished_rate_2017', 'cell_subscriptions_per100',
       'internet_per_mil_2017', 'military_exp_2017',
       'women_seats_percent_2017', 'male_bus_start_2017',
       'female_bus_start_2017', 'patent_apps_2017', 'sci_articles_2017',
       'pop_density_2017', 'HDI'],
      dtype='object')

In [4]:
X = df.drop(columns=['Country Name', 'Country Code', 'ffuel_energy_percent_2017', 'HDI'])
y = df['HDI']

In [5]:
lr = LinearRegression()

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [12]:
lr.fit(X_train, y_train)

LinearRegression()

In [13]:
lr.score(X_test, y_test)

0.9199083201247078